In [6]:
import json

with open("./drive/MyDrive/Dataset/Corona_NER/Corona2.json") as f:
  data = json.load(f)

print(data['examples'][0])

{'id': '18c2f619-f102-452f-ab81-d26f7e283ffe', 'content': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'metadata': {}, 'annotations': [{'id': '0825a1bf-

In [11]:
training_data = {'classes' : ['MEDICINE', "MEDICALCONDITION", "PATHOGEN"], 'annotations' : []}

for example in data["examples"]:
  temp_dict = {}
  temp_dict["entities"] = []
  temp_dict["text"] = example["content"]


  for annotation in example["annotations"]:
    start = annotation["start"]
    end = annotation["end"]
    label = annotation["tag_name"].upper()
    temp_dict["entities"].append((start,end,label))
  training_data["annotations"].append(temp_dict)

print(training_data["annotations"][0])

{'entities': [(360, 371, 'MEDICINE'), (383, 408, 'MEDICINE'), (104, 112, 'MEDICALCONDITION'), (679, 689, 'MEDICINE'), (6, 23, 'MEDICINE'), (25, 37, 'MEDICINE'), (461, 470, 'MEDICALCONDITION'), (577, 589, 'MEDICINE'), (853, 865, 'MEDICALCONDITION'), (188, 198, 'MEDICINE'), (754, 762, 'MEDICALCONDITION'), (870, 880, 'MEDICALCONDITION'), (823, 833, 'MEDICINE'), (852, 853, 'MEDICALCONDITION'), (461, 469, 'MEDICALCONDITION'), (535, 543, 'MEDICALCONDITION'), (692, 704, 'MEDICINE'), (563, 571, 'MEDICALCONDITION')], 'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in ch

In [12]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object to store docs

In [13]:
from spacy.util import filter_spans

for training_example in tqdm(training_data["annotations"]):
  text = training_example["text"]
  labels = training_example["entities"]
  doc = nlp.make_doc(text)
  ents = []

  for start, end, label in labels:
    span = doc.char_span(start, end, label=label, alignment_mode= "contract")

    if span is None:
      print("Skipping entity ")
    else:
      ents.append(span)
  filtered_ents = filter_spans(ents) # There might be a chance of indices of some entities to overlap, hence we use filter_spans to filter the entities
  doc.ents = filtered_ents
  doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the training data

100%|██████████| 31/31 [00:00<00:00, 318.54it/s]

Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 
Skipping entity 


In [21]:
!python -m spacy init fill-config base_config.cfg config.cfg

2023-09-24 03:39:39.543910: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [22]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

2023-09-24 03:39:56.405801: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    153.29    0.00    0.00    0.00    0.00
  7     200        734.85   3155.35   76.73   79.66   74.02    0.77
 14     400        669.55    734.48   97.24   97.24   97.24    0.97
 22     600        945.17    272.49   98.03   98.03   98.03    0.98
 30     800        188.78    219.05   98.23   98.04   98.43    0.98
 40    1000        224.41    192.51   98.41   99.20   97.64    0.98
 51    1200        206.75    185.68   98.24   97

In [23]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner("Antiretroviral therapy (ART) is recommended for all HIV-infected\
individuals to reduce the risk of disease progression.\nART also is recommended \
for HIV-infected individuals for the prevention of transmission of HIV.\nPatients \
starting ART should be willing and able to commit to treatment and understand the\
benefits and risks of therapy and the importance of adherence. Patients may choose\
to postpone therapy, and providers, on a case-by-case basis, may elect to defer\
therapy on the basis of clinical and/or psychosocial factors.")

colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)